In [1]:
import os
import random
import soundfile as sf
import audio_python as aus
import time
from tqdm import tqdm
from functools import partial

random.seed(42)
input_files = [
    os.path.join("/home/jmg/Music/Deadbeat/", f)
    for f in os.listdir("/home/jmg/Music/Deadbeat/")
    if f.endswith(".wav")
]
print(f"Found {len(input_files)} input files.")
print("Sample input files:", input_files[:5])

ITERS = 250  # increase if the flamegraph is too thin
DTYPE = "int16"

Found 12 input files.
Sample input files: ['/home/jmg/Music/Deadbeat/05 - Oblivion.flac.wav', '/home/jmg/Music/Deadbeat/02 - No Reply.flac.wav', '/home/jmg/Music/Deadbeat/09 - Ethereal Connection.flac.wav', '/home/jmg/Music/Deadbeat/11 - Afterthought.flac.wav', '/home/jmg/Music/Deadbeat/07 - Piece Of Heaven.flac.wav']


In [3]:
read_rust = aus.io.read_as_i16
read_py = partial(sf.read, dtype="int16")

def read_many(fn, path, iters):
    """
    Execute the hot path many times in one Python call.

    This maximises native execution time per sample and
    produces clean, meaningful flamegraphs.
    """
    timings = []
    for _ in tqdm(range(iters)):
        start = time.perf_counter()
        fn(path)
        end = time.perf_counter()
        timings.append(end - start)
    total_time = sum(timings)
    avg_time = total_time / iters
    return total_time, avg_time, timings

INPUT_FILE = "../test_output.wav" 
# input_files[
#         input_files.index("/home/jmg/Music/Deadbeat/09 - Ethereal Connection.flac.wav")
# ]
print(f"Using input file: {INPUT_FILE}")
for _ in range(10):
    read_rust(INPUT_FILE)
    read_py(INPUT_FILE)

print("Running Rust-backed reader...")
total_rs, avg_rs, timings_rs = read_many(read_rust, INPUT_FILE, ITERS)
print(
    f"Rust reader: total time {total_rs:.4f}s, avg time {avg_rs*1000:.4f}ms over {ITERS} iters"
)
print("Running soundfile reader...")
total_py, avg_rs, timings_py = read_many(read_py, INPUT_FILE, ITERS)

print(
    f"Python reader: total time {total_rs:.4f}s, avg time {avg_rs*1000:.4f}ms over {ITERS} iters"
)

Using input file: ../test_output.wav
Running Rust-backed reader...


100%|██████████| 250/250 [00:03<00:00, 76.92it/s]


Rust reader: total time 3.2404s, avg time 12.9617ms over 250 iters
Running soundfile reader...


100%|██████████| 250/250 [00:01<00:00, 173.98it/s]


Python reader: total time 3.2404s, avg time 5.7268ms over 250 iters


In [ ]:
def bench_function(f, input_generator, warmup=10, runs=100):
    # Warmup
    for _ in range(warmup):
        f(input_generator())

    timings = []

    for _ in tqdm(range(runs), desc="Run"):
        start_time = time.time()
        f(input_generator())
        end_time = time.time()
        timings.append(end_time - start_time)

    total_time = sum(timings)
    avg_time = total_time / runs
    print(f"Average time over {runs} runs: {avg_time:.6f} seconds")



def input_generator():
    return random.choice(input_files)

read_fn = aus.io.read_as_i16
# read_fn(input_generator())  # test run

for i in range(3):
    bench_function(read_fn, input_generator)


read_fn = partial(
    sf.read, dtype="int16"
)  # to match the file type, audio_python does this by default 👀

bench_function(read_fn, input_generator)

In [ ]:
import subprocess
command = "ffprobe -hide_banner -i '/home/jmg/Music/Deadbeat/09 - Ethereal Connection.flac.wav'"
# run and get the output
output = subprocess.check_output(command, shell=True).decode()
print("ffprobe output:", output)

# AudioPython

In [ ]:
data = read_rust(INPUT_FILE).to_mono(method="average", weights=None)
print("Data:", data)

aus.io.save("test_output.wav", data)